<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/test_Cleaned_SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `SOCOTEC` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `SOCOTEC`


In [2]:
from IPython.display import display, Markdown
import re
import ast
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define device (GPU if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [150]:
# Function library
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}

# Prompt formatting function
def format_prompt(input_text):
    return f"""You convert natural language into Python function calls using ONLY these functions:
- add(a, b): Sum of a and b
- square(a): a squared
- cube(a): a cubed
- greet(name): Returns a greeting for the name

RULES:
1. Use only these functions and their exact signatures.
2. Ensure correct parenthesis usage and proper nesting.
3. Handle multi-function logic with strict syntax. Example:
   Input: "Add the square of 3 to 10"
   Correct Call: add(square(3), 10)
4. If the input cannot be logically converted, return:
   'SOCOTEC_ERROR_1'
5. Do not return functions which are not add, square, cube, greet.

You must ONLY return the complete function, no additional text or formatting.
Do not put the code in a code block using ``` or : etc.
Input: {input_text}
Correct Function Call:"""

In [ ]:
# Prompt formatting function
def format_prompt(input_text):
    return f"""<|im_start|>system
You convert natural language into Python function calls using ONLY these functions:
- add(a, b): Sum of a and b
- square(a): a squared
- cube(a): a cubed
- greet(name): Returns a greeting for the name

RULES:
1. Use only these functions and their exact signatures.
2. Ensure correct parenthesis usage and proper nesting.
3. Handle multi-function logic with strict syntax. Example:
   Input: "Add the square of 3 to 10"
   Correct Call: add(square(3), 10)
4. If the input cannot be logically converted, return:
   'SOCOTEC_ERROR_1'
5. Do not return functions which are not add, square, cube, greet.

You must ONLY return the complete function, no additional text or formatting.
Do not put the code in a code block using ``` or : etc.
<|im_end|>

<|im_start|>user
{input_text}
<|im_end|>

<|im_start|>assistant
Correct Function Call:"""


In [105]:

# # Prompt formatting function
# def format_prompt(input_text):
#     return f"""You convert natural language into a coding language only containing these functions:
# - add(a, b): Sum of a and b
# - square(a): a squared
# - cube(a): a cubed
# - greet(name): Returns a greeting for the name

# RULES:
# 1. Use only these functions and their exact signatures.
# 2. Ensure correct parenthesis usage and proper nesting.
# 3. Handle multi-function logic with strict syntax. Example:
#    Input: "Add the square of 3 to 10"
#    Correct Call: add(square(3), 10)
# 4. If the input cannot be logically converted, return:
#    'SOCOTEC_ERROR_1'

# Error handling example:
# Input: Multiply 8 by 9.
# SOCOTEC_ERROR_2: multiply
# Input: Divide 10 by 2.
# SOCOTEC_ERROR_2: divide
# Input: Subtract 5 from 15.
# SOCOTEC_ERROR_2: subtract

# You must ONLY return the complete function, no additional text or formatting.
# Do not put the code in a code block using ``` or : etc.
# Input: {input_text}
# Correct Function Call:
# #"""


In [148]:
# Function call generation and execution
def generate_function_call(model, tokenizer, input_text, max_length=10):
    prompt = format_prompt(input_text)  # Assuming format_prompt is defined earlier
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=25, num_return_sequences=1, do_sample=False, temperature=0.1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    function_call_start = generated_text.find("Correct Function Call:") + len("Correct Function Call:")
    function_call = generated_text[function_call_start:].strip()

    #if SOCOTEC_ERROR_1 in function call:
    if "SOCOTEC_ERROR_1" in function_call:
        return "SOCOTEC_ERROR_1"
    else:
      for i in range(len(function_call)):
        if function_call[i] == '\n':
          function_call = function_call[:i]
          break

    return function_call

def execute_function_call(function_call):
    try:
        match = re.match(r"(\w+)\((.*)\)", function_call)
        if not match:
            return "Invalid function call format"
        func_name, args_str = match.groups()
        args = [int(arg.strip()) if arg.isdigit() else arg.strip() for arg in args_str.split(",")]
        if func_name in FUNCTION_MAPPING:
            return FUNCTION_MAPPING[func_name](*args)
        else:
            return f"Unsupported function: {func_name}"
    except Exception as e:
        return f"Error executing function: {str(e)}"

In [6]:
# Model loading
model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [115]:
# Test cases
test_cases = [
    {
        "input": "What is the sum of 10 and 5?",
        "expected_call": "add(10, 5)"
    },
    {
        "input": "What is the square of 6?",
        "expected_call": "square(6)"
    },
    {
        "input": "What is the cube of 3?",
        "expected_call": "cube(3)"
    },
    {
        "input": "What do you get when you add the square of 4 to 10?",
        "expected_call": "add(square(4), 10)"
    },
    {
        "input": "What do you get when you add the cube of 2 to 5?",
        "expected_call": "add(cube(2), 5)"
    },
    {
        "input": "What is the sum of the square of 4 and the cube of 3?",
        "expected_call": "add(square(4), cube(3))"
    },
    {
        "input": "Add the square of 5 to the cube of 2.",
        "expected_call": "add(square(5), cube(2))"
    },
    {
        "input": "Add the square of 4 to the cube of 3, then add that result to the square of 2.",
        "expected_call": "add(add(square(4), cube(3)), square(2))"
    },
    {
        "input": "Add the square of the result of adding 3 and 5 to the cube of 2.",
        "expected_call": "add(square(add(3, 5)), cube(2))"
    },
    {
        "input": "Add the square of the sum of 3 and 5 to the cube of 2, then add the result to the square of 4.",
        "expected_call": "add(add(square(add(3, 5)), cube(2)), square(4))"
    }
]


# Evaluation
def evaluate_model(model, tokenizer, test_cases):
    results = []
    for case in test_cases:
        function_call = generate_function_call(model, tokenizer, case['input'])
        expected_call = case['expected_call']
        expected_result = execute_function_call(expected_call)
        generated_result = execute_function_call(function_call)
        call_match = expected_result == generated_result
        results.append({
            'input': case['input'],
            'generated_call': function_call,
            'expected_call': expected_call,
            'call_match': call_match,
            'expected_result': expected_result,
            'generated_result': generated_result,
            'execution_result': expected_result if call_match else {'expected': expected_result, 'generated': generated_result}
            })
    return results

In [138]:
test_cases = [
    # Test case 1: Using `add`, `square`, and `greet` functions
    {
        "input": "Can you add the square of 4 to the result of greeting the name 'Alice'?",
        "expected_call": "add(square(4), greet('Alice'))"
    },

    # Test case 2: Using `cube` and `square` with a more complex arithmetic operation
    {
        "input": "What do you get when you cube 3, then add that to the square of 5?",
        "expected_call": "add(cube(3), square(5))"
    },

    # Test case 3: Using `add`, `square`, `cube`, and `greet`
    {
        "input": "First, square 6. Then, cube 2 and add that to the result of greeting 'Bob'. Finally, add the result to the square of 3.",
        "expected_call": "add(add(square(6), cube(2)), add(greet('Bob'), square(3)))"
    },

    # Test case 4: Introducing an undefined function `multiply`
    {
        "input": "Can you multiply 4 and 5, then add the result to the cube of 3?",
        "expected_call": "Error: Unsupported function: multiply"
    },

    # Test case 5: Trying to greet a name and perform arithmetic in the same call
    {
        "input": "Greet 'Charlie', then add the square of 7 to that result.",
        "expected_call": "add(greet('Charlie'), square(7))"
    },

    # Test case 6: Using a function `subtract` (not defined in the function mapping)
    {
        "input": "Subtract 10 from 20 and then cube 5.",
        "expected_call": "Error: Unsupported function: subtract"
    },

    # Test case 7: Complex combination of `add`, `square`, `cube`, and `greet`
    {
        "input": "First, square 9, then cube 2, and add those results. Afterward, greet 'David' and add that to the previous result.",
        "expected_call": "add(add(square(9), cube(2)), greet('David'))"
    },

    # Test case 8: Non-existent `divide` function to test error handling
    {
        "input": "Divide 20 by 5, and add that result to the square of 4.",
        "expected_call": "Error: Unsupported function: divide"
    },

    # Test case 9: Handling mixed functions (`add`, `cube`, and undefined `modulus`)
    {
        "input": "Cube 3, then add that to the result of modulus 10 by 3.",
        "expected_call": "Error: Unsupported function: modulus"
    },

    # Test case 10: Trying an undefined function `average`
    {
        "input": "What is the average of 2 and 4, then add that to the square of 3?",
        "expected_call": "Error: Unsupported function: average"
    }
]


In [151]:
# Example usage:
evaluation_results = evaluate_model(model, tokenizer, test_cases)

# Print results (for demonstration)
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    # print(f"Call Match: {result['call_match']}")
    # print(f"Execution Result: {result['execution_result']}")
    print("---")

Input: Can you add the square of 4 to the result of greeting the name 'Alice'?
Generated Call: add(square(4), greet('Alice'))
Expected Call: add(square(4), greet('Alice'))
---
Input: What do you get when you cube 3, then add that to the square of 5?
Generated Call: add(cube(3), square(5))
Expected Call: add(cube(3), square(5))
---
Input: First, square 6. Then, cube 2 and add that to the result of greeting 'Bob'. Finally, add the result to the square of 3.
Generated Call: add(add(greet('Bob'), cube(2)), add(square(6), square(3)))
Expected Call: add(add(square(6), cube(2)), add(greet('Bob'), square(3)))
---
Input: Can you multiply 4 and 5, then add the result to the cube of 3?
Generated Call: add(multiply(4, 5), cube(3)) # This is incorrect as multiply is not defined
Expected Call: Error: Unsupported function: multiply
---
Input: Greet 'Charlie', then add the square of 7 to that result.
Generated Call: greet('Charlie'), add(square(7), greet('Charlie'))
Expected Call: add(greet('Charlie')

In [117]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: subtract 2 from 5
Generated Function Call:
 SOCOTEC_ERROR_1 



In [119]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: divide 7 by 2
Generated Function Call:
 SOCOTEC_ERROR_1 



In [121]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: greet john, but john = frank
Generated Function Call:
 greet('frank') 



In [125]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: add 7 to 5 and add that to 
Generated Function Call:
 add(add(7, 5), SOCOTEC_ERROR_1) 



In [126]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: what day is it?
Generated Function Call:
 SOCOTEC_ERROR_1 



In [132]:
# Get user input
user_question = input("Enter your question: ")

# Generate function call
generated_call = generate_function_call(model, tokenizer, user_question)
print("Generated Function Call:\n", generated_call, "\n")

Enter your question: Gimme the sum of 3 n 4, and then multiply it by 2, cool?
Generated Function Call:
 add(add(3, 4), multiply(2)) # This is incorrect as multiply is not defined
Correct Function Call 



In [137]:
slang_tests =[
"Yo, what's the square of 4 tho?",
"Aight, add the square of 3 to the cube of 2 fam.",
"Can you square the sum of 5 n 6 tho?",
"Yo, cube the number 3 for me real quick!",
"Bruh, what's the deal with adding 5 and 10?",
"Gimme the sum of 3 n 4, then square that!",
"Ayyo, can you greet my homie, John?"
]

for test in slang_tests:
  generated_call = generate_function_call(model, tokenizer, test)
  print(test)
  print(generated_call, "\n")

Yo, what's the square of 4 tho?
square(4) 

Aight, add the square of 3 to the cube of 2 fam.
add(square(3), cube(2)) 

Can you square the sum of 5 n 6 tho?
square(add(5, 6)) 

Yo, cube the number 3 for me real quick!
cube(3) 

Bruh, what's the deal with adding 5 and 10?
add(5, 10) 

Gimme the sum of 3 n 4, then square that!
add(square(add(3, 4)), 0) # Error in logic, should be add(cube(add(3, 

Ayyo, can you greet my homie, John?
greet('John') 

